In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import math

In [ ]:
def openfile(fileName):
    """
    Dado el nombre de un fichero devuelve la representación y las etiquetas
    """
    with open(fileName,"rb") as fh:
        representacion,etiquetas = pickle.load(fh)
    return representacion,etiquetas

In [ ]:
#Cargamos las mejores representaciones de cada
ncharTrain,charetiquetasTrain = openfile("ncharRepTrain")
ncharTest,charetiquetasTest = openfile("ncharRepTest")
print("n-gramas CHAR BOW --- OK")
w2vTrain,w2vetiquetasTrain = openfile("w2vRepTrain")
w2vTest,w2vetiquetasTest = openfile("w2vRepTest")
print("W2V REPR ------ OK")
pcarasTrain,pcarasetiquetasTrain = openfile("pcarasTrain")
pcarasTest,pcarasetiquetasTest = openfile("pcarasTest")
pcarasetiquetasTrain = [-1 if g == "male" else 1  for g in pcarasetiquetasTrain]
pcarasetiquetasTest = [-1 if g == "male" else 1  for g in pcarasetiquetasTest]
print("Porcentaje de caras de hombres en fotos --- OK")
bovwTrain,bovwetiquetasTrain = openfile("bovwRepTrain")
bovwTest,bovwetiquetasTest = openfile("bovwRepTest")
bovwetiquetasTrain = [-1 if g == "male" else 1  for g in bovwetiquetasTrain]
bovwetiquetasTest = [-1 if g == "male" else 1  for g in bovwetiquetasTest]
print("Bag of Visual Words --- OK")

In [ ]:
#Validamos que las etiquetas esten correctas
train = all(a == b == c ==d for a,b,c,d in zip(charetiquetasTrain,w2vetiquetasTrain,pcarasetiquetasTrain,bovwetiquetasTrain))
test = all(a == b == c == d for a,b,c,d in zip(charetiquetasTest,w2vetiquetasTest,pcarasetiquetasTest,bovwetiquetasTest))
print("Train: %r Test: %r " %(train,test))

In [ ]:
#Entreno modelo Char-ngram
nchar_clf = LogisticRegression(C=10)
nchar_clf.fit(ncharTrain,charetiquetasTrain)
print("OK! %.3f" % nchar_clf.score(ncharTest,charetiquetasTest))

In [ ]:
#Entreno modelo Word2Vec
w2v_clf = svm.SVC(C = 1000,kernel="linear")
w2v_clf.fit(w2vTrain,w2vetiquetasTrain)
print("OK! %.3f" %w2v_clf.score(w2vTest,w2vetiquetasTest))

In [ ]:
#Entreno modelo % hombres en fotos
pcaras_clf = svm.SVC(C = 1,kernel="rbf")
pcaras_clf.fit(np.array(pcarasTrain).reshape((3000,1)),pcarasetiquetasTrain)
print("OK! %.3f" %pcaras_clf.score(np.array(pcarasTest).reshape((2200,1)),pcarasetiquetasTest))

In [ ]:
#Entreno modelo basado en BOVW con VGG y reducida a 20 dimensiones con PCA
bovw_clf = LogisticRegression(C=1)
bovw_clf.fit(bovwTrain,bovwetiquetasTrain)
print("OK! %.3f" % bovw_clf.score(bovwTest,bovwetiquetasTest))

In [ ]:
#Construyo una matriz con las predicciones sobre el conjunto de test 
results = np.zeros((2200,4))
results[:,0] = nchar_clf.predict(ncharTest)
results[:,1] = w2v_clf.predict(w2vTest)
results[:,2] = pcaras_clf.predict(np.array(pcarasTest).reshape((2200,1)))
results[:,3] = bovw_clf.predict(bovwTest)


In [ ]:
#Aplico votación mayoritaria si la suma por fila <0 --> Hombre sino --> Mujer (Empate mujer)
results_procesados = [-1 if val < 0 else 1 for val in list(results.sum(1))] #suma por filas

In [ ]:
#Calculo la probabilidad de acertar
pacierto = sum([1 if valpredecido == etiqueta else 0 for valpredecido,etiqueta in zip(results_procesados,charetiquetasTest)]) / len(results_procesados)
print(pacierto)
